In [ ]:
import os
import scprep
import demap
import math
import random
import numpy as np
import pandas as pd
import hierarchical_umap as h_umap
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.utils import check_array
from sklearn.preprocessing import normalize, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from sklearn.neighbors import NearestNeighbors

from scipy.stats import pearsonr, spearmanr

In [15]:
def correlation(X, X_emb):
    
    dist_orig = np.square(euclidean_distances(X, X)).flatten()
    dist_emb = np.square(euclidean_distances(X_emb, X_emb)).flatten()
    
    
    coef, p = spearmanr(dist_orig, dist_emb)
    return coef

def stress(X, X_emb):
    
    DE = euclidean_distances(X_emb)
    DE = DE/np.max(DE)
    DH = euclidean_distances(X)
    DH = DH/np.max(DH)
    stress = 0.5 * np.sum((DE - DH)**2)
    
    return np.sqrt(stress/(0.5*np.sum(DH**2)))
    

def neighborhood_preservation(X, X_emb, Khigh=30):
    
    neigh_high = NearestNeighbors(n_neighbors=Khigh+1, n_jobs=-1)
    neigh_high.fit(X)
    high_dists, high_indices = neigh_high.kneighbors(X)


    neigh_emb = NearestNeighbors(n_neighbors=Khigh+1, n_jobs=-1)
    neigh_emb.fit(X_emb)
    emb_dists, emb_indices = neigh_emb.kneighbors(X_emb)

    npres = np.zeros(Khigh)
    
    for k in range(1, Khigh+1):
        for i in range(X.shape[0]):
            high_current = high_indices[i][1:k+1]
            emb_current = emb_indices[i][1:k+1]
            
            tp = len(np.intersect1d(high_current, emb_current))
            
            npres[k-1] += (tp/k)
        
        
    npres /= float(X.shape[0])
    
    return npres

In [16]:
def get_fmnist():
    fashionTrain = pd.read_csv('data/fashion-train.csv')

    fashionX = fashionTrain.values[:,2:]
    fashionY = fashionTrain.values[:, 1].astype(int)

    X = normalize(fashionX)
    y = fashionY

    X = check_array(X, dtype=np.float32, accept_sparse='csr', order='C')
    
    return X, y

def get_mnist():
    X = np.load('./data/MNIST_70000.npy')
    y = np.load('./data/MNIST_70000_label.npy').astype(int)
    X = normalize(X)
    X = check_array(X, dtype=np.float32, accept_sparse='csr', order='C')
    
    return X, y

def load_scRNAseq():
    download_path = os.path.expanduser("~/Documentos/HierarchicalUMAP/umap-cpp/umap/cpp/data")
    sparse=True
    T1 = scprep.io.load_10X(os.path.join(download_path, "scRNAseq", "T0_1A"), sparse=sparse, gene_labels='both')
    T2 = scprep.io.load_10X(os.path.join(download_path, "scRNAseq", "T2_3B"), sparse=sparse, gene_labels='both')
    T3 = scprep.io.load_10X(os.path.join(download_path, "scRNAseq", "T4_5C"), sparse=sparse, gene_labels='both')
    T4 = scprep.io.load_10X(os.path.join(download_path, "scRNAseq", "T6_7D"), sparse=sparse, gene_labels='both')
    T5 = scprep.io.load_10X(os.path.join(download_path, "scRNAseq", "T8_9E"), sparse=sparse, gene_labels='both')
    print("done 1")
    filtered_batches = []
    for batch in [T1, T2, T3, T4, T5]:
        batch = scprep.filter.filter_library_size(batch, percentile=20, keep_cells='above')
        batch = scprep.filter.filter_library_size(batch, percentile=75, keep_cells='below')
        filtered_batches.append(batch)
    del T1, T2, T3, T4, T5
    print("done 2")
    EBT_counts, sample_labels = scprep.utils.combine_batches(
        filtered_batches, 
        ["Day 00-03", "Day 06-09", "Day 12-15", "Day 18-21", "Day 24-27"],
        append_to_cell_names=True
    )
    del filtered_batches # removes objects from memory
    print("done 3")
    EBT_counts = scprep.filter.filter_rare_genes(EBT_counts, min_cells=10)
    EBT_counts = scprep.normalize.library_size_normalize(EBT_counts)
    mito_genes = scprep.select.get_gene_set(EBT_counts, starts_with="MT-") # Get all mitochondrial genes. There are 14, FYI.
    EBT_counts, sample_labels = scprep.filter.filter_gene_set_expression(
    EBT_counts, sample_labels, genes=mito_genes, 
    percentile=90, keep_cells='below')
    print("done 4")
    EBT_counts = scprep.transform.sqrt(EBT_counts)
    
    le = LabelEncoder()
    le.fit(sample_labels)
    labels = le.transform(sample_labels)
    print("done 5")
    X = PCA(n_components=100).fit_transform(EBT_counts.values)
    X = check_array(X, dtype=np.float32, accept_sparse='csr', order='C')
    return X, labels

def load_mammals():
    X = np.loadtxt("data/mammals-20000_features.txt")
    y = np.loadtxt("data/mammals-20000_classes.txt")
    X = normalize(X)
    
    return X, y

X, y = load_scRNAseq()  
# X, y = load_mammals()


# reducer = h_umap.HUMAP('precomputed', np.array([0.20, 0.19]), 15, 0.05, 'NNDescent', 0.0, True)

# reducer.set_distance_similarity(False)
# reducer.set_path_increment(False)
# reducer.set_landmarks_nwalks(10)
# reducer.set_landmarks_wl(5)
# reducer.set_influence_nwalks(10)
# reducer.set_influence_wl(10)
# inf_neighborhood = int(0.1*15)
# print("inf_neighborhood", inf_neighborhood)
# reducer.set_influence_neighborhood(inf_neighborhood)

# reducer.fit(X, y)
# embedding_2 = reducer.get_embedding(2)
# y_2 = reducer.get_labels(2)
# X_2 = X[reducer.get_original_indices(2), :]

# indices_2 = random.sample(range(0, len(y_2)), min(3000, len(y_2)))
# subset_emb_2 = embedding_2[indices_2]
# subset_X_2 = X_2[indices_2]

# plt.scatter(embedding_2[:, 0], embedding_2[:, 1], c=y_2, alpha=0.6, cmap='Spectral')

done 1
done 2
done 3
done 4
done 5


In [17]:
# distance_similarities = [False, True]
# path_increments = [False, True]
# n_neighbors =  [15, 20, 40, 50, 70, 90, 100]
# landmarks_nwalks = [10, 20, 30]
# landmarks_walklengths = [5, 10, 20, 30, 50, 100]
# influence_nwalks =  [10, 20, 30]
# influence_walklengths = [10, 30, 50, 70, 80, 90, 100]
# influence_neighborhoods = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 1]
# min_dists = [0.05,0.1, 0.15]
distance_similarities = [False, True]
path_increments = [False, True]
n_neighbors =  [15, 30,  60,  100]
landmarks_nwalks = [10]
landmarks_walklengths = [10]
influence_nwalks =  [20]
influence_walklengths = [30]
influence_neighborhoods = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9, 1]
min_dists = [0.15]
executions = 10

n_parameters = len(distance_similarities) * len(path_increments) * len(n_neighbors) * len(landmarks_nwalks) * len(influence_neighborhoods) 
print("%d parameters." % (n_parameters))

128 parameters.


In [18]:
id_conf = 0
file_conf = open('experiments/humap_parameters/current_analysis.csv', 'w')
file_conf.write('id,distance_similarity,path_increment,n_neighbors,landmarks_nwlandmarks_wl,influence_nw,influence_wl,influence_neighborhood,n_dist\n')
file_conf.close()

for distance_similarity in distance_similarities:
    for path_increment in path_increments:
        for n_neigh in n_neighbors:
            for landmarks_nwalk in landmarks_nwalks:
                for landmarks_walklength in landmarks_walklengths:
                    for influence_nwalk in influence_nwalks:
                        for influence_walklength in influence_walklengths:
                            for influence_neighborhood in influence_neighborhoods:
                                for min_dist in min_dists:
                                    file_conf = open('experiments/humap_parameters/current_analysis.csv', 'a')
                                    file_conf.write(str(id_conf)+','+str(distance_similarity)+','+str(path_increment)+','+str(n_neigh)+','+str(landmarks_nwalk)+','+str(landmarks_walklength)+','+str(influence_nwalk)+','+str(influence_walklength)+','+str(influence_neighborhood)+','+str(min_dist)+'\n')
                                    file_conf.close()
                                    inf_neighborhood = int(influence_neighborhood*n_neigh)
                                    path = 'experiments/humap_parameters/'+str(id_conf)+'/'
                                    id_conf += 1
                                    os.mkdir(path)
                                    
                                    correlation_list = []
                                    np_list = []
                                    demap_list = []
                                    
                                    for i in range(executions):

                                        reducer = h_umap.HUMAP('precomputed', np.array([0.20, 0.19]), n_neigh, min_dist, 'NNDescent', 0.0, True)

                                        reducer.set_distance_similarity(distance_similarity)
                                        reducer.set_path_increment(path_increment)
                                        reducer.set_landmarks_nwalks(landmarks_nwalk)
                                        reducer.set_landmarks_wl(landmarks_walklength)
                                        reducer.set_influence_nwalks(influence_nwalk)
                                        reducer.set_influence_wl(influence_walklength)
                                        reducer.set_influence_neighborhood(inf_neighborhood)

                                        reducer.fit(X, y)
                                        embedding_2 = reducer.get_embedding(2)
                                        y_2 = reducer.get_labels(2)
                                        X_2 = X[reducer.get_original_indices(2), :]
                                        
                                        indices_2 = random.sample(range(0, len(y_2)), min(3000, len(y_2)))
                                        subset_emb_2 = embedding_2[indices_2]
                                        subset_X_2 = X_2[indices_2]
                                        
                                        plt.scatter(embedding_2[:, 0], embedding_2[:, 1], c=y_2, alpha=0.6, cmap='Spectral')
                                        plt.savefig(path+'/embedding_'+str(i)+'.svg')
                                        plt.clf()
                                        
                                        demap_value = demap.DEMaP(subset_X_2, subset_emb_2)
                                        demap_list.append(demap_value)
                                        corr_value = correlation(subset_X_2, subset_emb_2)
                                        correlation_list.append(corr_value)
                                        npres = neighborhood_preservation(subset_X_2, subset_emb_2)
                                        np_list = np_list + npres.tolist()
                                    
                                    df_corr = pd.DataFrame({
                                        'index': list(range(len(correlation_list))),
                                        'corr': correlation_list
                                    })
                                    
                                    df_np = pd.DataFrame({
                                        'n_neighbors': list(range(30))*executions,
                                        'n_preservation': np_list
                                    })
                                    
                                    df_demap = pd.DataFrame({
                                        'index': list(range(len(demap_list))),
                                        'demap': demap_list
                                    })
                                    print("\n"+str(id_conf)+','+str(distance_similarity)+','+str(path_increment)+','+str(n_neigh)+','+str(landmarks_nwalk)+','+str(landmarks_walklength)+','+str(influence_nwalk)+','+str(influence_walklength)+','+str(influence_neighborhood)+','+str(min_dist)+'\n')
                                    print("correlation: %.4f +/- %.4f" % (np.mean(correlation_list), np.std(correlation_list)))
                                    print("demap: %.4f +/- %.4f" % (np.mean(demap_list), np.std(demap_list)))
                                    
                                    df_corr.to_csv(path+'/correlation.csv', index=False)
                                    df_np.to_csv(path+'/npreservation.csv', index=False)
                                    df_demap.to_csv(path+'/demap.csv', index=False)
                                    

                                    
                                    
                                        
                                        
                                        
                                        



1,False,False,15,10,10,20,30,0.0,0.15

correlation: 0.4900 +/- 0.0771
demap: 0.5976 +/- 0.0571

2,False,False,15,10,10,20,30,0.1,0.15

correlation: 0.5146 +/- 0.0394
demap: 0.6307 +/- 0.0425

3,False,False,15,10,10,20,30,0.2,0.15

correlation: 0.4769 +/- 0.0732
demap: 0.5949 +/- 0.0565

4,False,False,15,10,10,20,30,0.3,0.15

correlation: 0.5120 +/- 0.0715
demap: 0.6298 +/- 0.0572

5,False,False,15,10,10,20,30,0.5,0.15

correlation: 0.4908 +/- 0.0654
demap: 0.5930 +/- 0.0431

6,False,False,15,10,10,20,30,0.7,0.15

correlation: 0.4868 +/- 0.0642
demap: 0.5981 +/- 0.0629

7,False,False,15,10,10,20,30,0.9,0.15

correlation: 0.5297 +/- 0.0516
demap: 0.6149 +/- 0.0437

8,False,False,15,10,10,20,30,1,0.15

correlation: 0.5451 +/- 0.0515
demap: 0.6177 +/- 0.0457

9,False,False,30,10,10,20,30,0.0,0.15

correlation: 0.5426 +/- 0.0459
demap: 0.6612 +/- 0.0303

10,False,False,30,10,10,20,30,0.1,0.15

correlation: 0.5451 +/- 0.0576
demap: 0.6509 +/- 0.0481

11,False,False,30,10,10,20,30,0.2,0.15




87,True,False,60,10,10,20,30,0.9,0.15

correlation: 0.4758 +/- 0.0753
demap: 0.6032 +/- 0.0669

88,True,False,60,10,10,20,30,1,0.15

correlation: 0.4838 +/- 0.0453
demap: 0.6030 +/- 0.0572

89,True,False,100,10,10,20,30,0.0,0.15

correlation: 0.6551 +/- 0.0216
demap: 0.7614 +/- 0.0139

90,True,False,100,10,10,20,30,0.1,0.15

correlation: 0.6470 +/- 0.0151
demap: 0.7549 +/- 0.0143

91,True,False,100,10,10,20,30,0.2,0.15

correlation: 0.6076 +/- 0.0376
demap: 0.7420 +/- 0.0328

92,True,False,100,10,10,20,30,0.3,0.15

correlation: 0.5662 +/- 0.0427
demap: 0.7050 +/- 0.0314

93,True,False,100,10,10,20,30,0.5,0.15

correlation: 0.5184 +/- 0.0605
demap: 0.6475 +/- 0.0476

94,True,False,100,10,10,20,30,0.7,0.15

correlation: 0.5324 +/- 0.0547
demap: 0.6508 +/- 0.0302

95,True,False,100,10,10,20,30,0.9,0.15

correlation: 0.5205 +/- 0.0653
demap: 0.6282 +/- 0.0664

96,True,False,100,10,10,20,30,1,0.15

correlation: 0.5390 +/- 0.0354
demap: 0.6478 +/- 0.0397

97,True,True,15,10,10,20,30,0.0,0.1

<Figure size 432x288 with 0 Axes>

In [22]:
## arrumei a distancia dos elementos...
# agora posso tentar utilizar aproximação por landmarks